In [1476]:
import os
import sys
import re
import pandas as pd
import numpy as np
from enum import Enum
import zipfile
from typing import Optional, Union, List, Callable, Dict
from dataclasses import dataclass, make_dataclass, field
from abc import ABC

PKG_ROOT = os.path.dirname(os.path.realpath(os.getcwd()))
if not PKG_ROOT in sys.path:
    sys.path.append(PKG_ROOT)

In [1477]:
class IDS:
    
    @classmethod
    def make_names_old(cls, name:str, df:pd.DataFrame):
        '''
        Construct dataclass or nested dataclass from a dataframe
        '''
        if df.shape[1] == 2:
            data = [(_[1], str, field(default=_[0])) for _ in df.values]
            return make_dataclass(name, data)
        else:
            group = df.groupby(df.columns[0])
            data = [(k, dataclass, field(default=cls.make_names_old(k, v.drop(v.columns[0], axis=1)))) for k, v in group]
            return make_dataclass(name, data)

    @classmethod
    def make_names_(cls, name:str, df:pd.DataFrame):
        '''
        Construct dataclass or nested dataclass from a dataframe
        '''
        if df.shape[1] == 2:
            data = [(_[1], str, field(default=_[0])) for _ in df.values]
            return make_dataclass(name, data)
        else:
            group = df.groupby(df.columns[0])
            if group.ngroups == 1:
                return cls.make_names_(name, df.drop(df.columns[0], axis=1))
            data = [(k, dataclass, field(default=cls.make_names_(k, v.drop(v.columns[0], axis=1)))) for k, v in group]
            return make_dataclass(name, data)
        
    @classmethod
    def make_names(cls, name:str, df:pd.DataFrame, ignoreCols:List=[]):
        return cls.make_names_(name=name, df=df[[_ for _ in df.columns if not _ in ignoreCols]])

    @staticmethod
    def make_enum(name:str, df:pd.DataFrame, nameCol:int=1, valueCol:Optional[Union[int, list]]=None):
        if df.shape[1] == 1:
            df = df.reset_index()
        nameCol = df.columns[nameCol]
        if valueCol is None:
            valueCol = df.columns.difference([nameCol])
        else:
            valueCol = df.columns[valueCol]
        if len(valueCol) == 1:
            valueCol = valueCol[0]
        if isinstance(valueCol, (list, tuple, pd.Index)):
            return Enum(name, df.set_index(nameCol)[valueCol].apply(tuple, axis=1).to_dict())
        else:
            return Enum(name, df.set_index(nameCol)[valueCol].to_dict())
        
    @staticmethod
    def make_map(df, cols:List):
        d1 = df[df.columns[cols[:2]]].set_index(df.columns[cols[0]]).drop_duplicates().to_dict()
        d2 = df[df.columns[cols[:2]]].set_index(df.columns[cols[1]]).drop_duplicates().to_dict()
        d1.update(d2)
        return d1

In [1478]:
def clean_duplicates(df:pd.DataFrame, groupby:Optional[Union[List, str]]=None):
    if groupby is None:
        col = df.columns[-1]
        df = df.groupby(col).apply(lambda x: x.reset_index())
        if df.index.nlevels > 1:
            df = df.droplevel(0)
        df[col] = df.apply(lambda x: f'{re.sub("_+", "_", x[col])}{("_" + str(x.name)) if x.name > 0 else ""}', axis=1)
        return df.reset_index(drop=True).drop('index', axis=1)
    else:
        return df.groupby(groupby).apply(clean_duplicates).reset_index(drop=True)

In [1479]:
class BlockBase:
    
    @staticmethod
    def bytes2Num(barray:bytes, signed:bool=False, order:str='little'):
        return int.from_bytes(barray, signed=signed, byteorder=order)
    
    @staticmethod
    def num2Bytes(num:int, length:int, signed=False, order='little'):
        return num.to_bytes(length, signed=signed, byteorder=order)
    
    @staticmethod
    def bytes2Str(barray:bytes):
        ba = np.array(bytearray(barray))
        p = np.argmax(np.logical_or(ba > 127
                                   ,ba < 32))
        if p == 0: ## All strings will have >1 length so when p == 0, it only means the string is the whole length of the bytearray
            p = len(barray)
        return barray[:p].decode('latin')
    
    @staticmethod
    def to_int(num:float):
        '''
        Integer will be coerced to float if np.nan exists in the same column
        Force the value to int, if np.nan then return np.nan
        '''
        if isinstance(num, str): #0x0000 in dataframe
            return int(num, 16)
        if num == None:
            return None
        if np.isnan(num):
            return None
        return int(num)


In [1480]:
class ValueBlock(BlockBase):
    
    def __init__(self
                 ,pbuffer: bytearray  #contents of parent segment
                 ,valueLoc: int
                 ,size: int
                 ,signed: bool = False
                 ,order: str = 'little'
                 ,optimize: Optional[int] = None
                 ,skiphead: int = 0
                 ,skiptail: int = 0):
        self.pbuffer = pbuffer
        self.valueLoc = self.to_int(valueLoc)
        self.size = self.to_int(size)
        self.signed = signed
        self.order = order
        self.optimize = self.to_int(optimize)
        self.skiphead = skiphead
        self.skiptail = skiptail
        self.notVoid = self.not_void()
    
    def not_void(self):
        if any([pd.isnull(self.valueLoc), pd.isnull(self.size)]):
            return False
        if any([self.pbuffer is None
                ,len(self.pbuffer)==0
                ,self.valueLoc < 0
                ,self.size <= 0
                ,self.valueLoc + self.size > len(self.pbuffer)]):
            return False
        return True
    
    def bind_buffer(self, buffer:bytearray):
        self.pbuffer = buffer
        return self
     
    @property
    def bytes(self):
        if self.not_void:
            return self.pbuffer[self.valueLoc+self.skiphead
                               :self.valueLoc+self.size+self.skiphead-self.skiptail]
        return None
        
    @property
    def value(self):
        if self.not_void:
            return self.bytes2Num(self.bytes, self.signed, self.order)
        return None
    
    @property
    def string(self):
        if self.not_void:
            return self.bytes2Str(self.bytes)
        return None
    
    def set_value(self, value:Union[int, Enum]):            
        if hasattr(value, 'value'):
                value = value.value
        value = self.to_int(value)
        if self.notVoid and value is not None:
            self.pbuffer[self.valueLoc+self.skiphead
                         :self.valueLoc+self.size+self.skiphead-self.skiptail] = self.num2Bytes(value, self.size, self.signed, self.order) 
    
            
    def inc_value(self, inc_value:int=1):
        if self.notVoid:
            self.set_value(self.value + inc_value)
    

In [1481]:
class RecordBase(BlockBase):
     
    def __init__(self, segList:List, nameMap: Optional[Dict]=None, template:Optional[bytearray]=None):
        '''
        SegList has the format of [(name, length, datatype)]
        datatype includes: name, which will be mapped to codes in nameMap
                           int
                           str
                           None
        '''
        self.pattern = re.compile(b''.join([f'(?P<{n}>.{{{s}}})'.encode() for (n, s, t) in segList]))
        self.names, self.sizes, self.dtypes = zip(*segList)
        self.sizes = np.array(self.sizes)
        self.sizeMap = dict(zip(self.names, self.sizes))
        self.rangeMap = dict([(n, r) for (n, r) in zip(self.names, [range(*_) for _ in zip(np.concatenate([[0], self.sizes.cumsum()])
                                                                                           ,self.sizes.cumsum())])])
        self.typeMap = dict(zip(self.names, self.dtypes))
        self.size = self.sizes.sum()
        self.nameMap = nameMap
        if template is None:
            self.template = bytearray(b'0' * self.size)
        else:
            self.template = template
            
            
    def parse(self, buffer:bytearray):
        return pd.DataFrame(_.groupdict() for _ in re.finditer(self.pattern, buffer))
    
    def infer_col(self, col:pd.Series):
        dtype = self.typeMap.get(col.name, None)
        if dtype == None:
            return col
        if dtype == int:
            return col.apply(self.bytes2Num)
        if dtype == str:
            return col.apply(self.bytes2Str)
        if dtype.upper() == 'INTNAME':
            return col.apply(lambda x: self.nameMap(self.bytes2Num(x)).name)
        if dtype.upper() == 'STRNAME':
            return col.apply(lambda x: self.nameMap(self.bytes2Str(x)).name)
    
    def infer(self, df:pd.DataFrame):
        return pd.concat([self.infer_col(col) for name, col in df.iteritems()], axis=1)
    
    


In [1482]:
class TableBlock(ValueBlock):
    '''
    Create ValueBlock that using a table of values to fill
    refTable includes Kit, Race, Class, Gender, Alignment and RacialEnemy defined in CRETABLE.csv file
    '''
    
    def __init__(self
                ,pbuffer: bytearray
                ,valueLoc: int
                ,size: int
                ,signed: bool
                ,order: str
                ,refTableName: str
                ,optimize: Optional[int]
                ,refTable: Enum):
        super().__init__(pbuffer, valueLoc, size, signed, order, optimize)
        self.refTable = refTable
        
        
    def inc_value(self, inc_value:int=1):
        pass
    
    @property
    def value(self):
        v = super().value
        if v is not None:
            return self.refTable(v).name
        return None
    
    def set_value(self, value:Union[int, Enum, str]):
        if isinstance(value, str):
            for k, v in self.refTable.__members__.items():
                if value.upper() == k:
                    value = v.value
                    break
            else:
                value = None
        super().set_value(value) 

In [1483]:
class SegBlock(BlockBase):
    '''
    Base class of subStructures as defined in GAMSEGS.csv and CRESEGS.csv
    Used for dummy subStructures that will not be modified
    '''
    
    def __init__(self
                ,savRef: object
                ,parentRef: object
                ,pbuffer: bytearray
                ,name: str
                ,countLoc: int  #offset pointer to buffer
                ,offsetLoc: int  #count pointer to buffer
                ,sizeLoc: int
                ,sizeValue: Optional[int] = None
                ,countValue: Optional[int] = None
                ,offsetValue: Optional[int] = None):  #only the header segment has no offset value
        self.savRef = savRef
        self.parentRef = parentRef
        self.resourceDir = savRef.resourceDir
        self.name = name
        self.pbuffer = pbuffer
        self.sizeValue = self.to_int(sizeValue)
        self.countValue = self.to_int(countValue)
        self.offsetValue = self.to_int(offsetValue)
        self.offsetBlock = ValueBlock(self.pbuffer, self.to_int(offsetLoc), 4)
        self.countBlock = ValueBlock(self.pbuffer, self.to_int(countLoc), 4)
        self.sizeBlock = ValueBlock(self.pbuffer, self.to_int(sizeLoc), 4)
        self.buffer = self.pbuffer[self.offsetValue: self.offsetValue + self.sizeValue]
        self.previous = None

            
    def pack(self):
        self.sizeValue = len(self.buffer)
        if self.previous is not None:
            self.offsetValue = self.previous.offsetValue + self.previous.sizeValue
        return self.buffer
    

In [1484]:
class RecordsBlock(SegBlock):
    '''
    Parse subStructures in CRE files that's composed of a list of records
    including spell, item, effect, and item
    '''
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.countValue = self.countBlock.value
        self.df = self.Pattern.parse(self.buffer)
        
    def __repr__(self):
        display(self.Pattern.infer(self.df))
        return f'{self.__class__} {self.name}'
        

In [1485]:
class Party(SegBlock):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.CREIndexLength, self.CREOffsetLoc, self.CRESizeLoc = self.get_CRE_locs()
        self.parse_party()
        
    def get_CRE_locs(self):
        loc_df = self.savRef.creLocs
        CREIndexLen = self.to_int(loc_df[loc_df['SubSegs']=='Header']['SizeValue'].iloc[0])
        CREOffsetLoc, CRESizeLoc =  loc_df[loc_df['SubSegs']=='CRE'][['OffsetLoc', 'SizeLoc']].iloc[0].apply(self.to_int)
        return CREIndexLen, CREOffsetLoc, CRESizeLoc
    
    def parse_party(self):
        self.indexHeader = self.buffer[: self.CREIndexLength * self.countBlock.value]
        self.CREOffsetBlocks = [ValueBlock(self.buffer, self.CREOffsetLoc + self.CREIndexLength * i, 4) for i in range(self.countBlock.value)]
        self.CRESizeBlocks = [ValueBlock(self.buffer, self.CRESizeLoc + self.CREIndexLength * i, 4) for i in range(self.countBlock.value)]
        vRecords = self.savRef.creValues
        tRecords = self.savRef.creTables
        sRecords = self.savRef.creSegs
        self.CRES = [CRE(self.savRef
                        ,self
                        ,self.pbuffer[o.value: o.value + s.value]
                        ,vRecords
                        ,tRecords
                        ,sRecords
                        ) for o, s in zip(self.CREOffsetBlocks, self.CRESizeBlocks)]
        
    def pack(self):
        [_.pack() for _ in self.CRES]
        self.buffer = bytearray(b''.join([self.indexHeader, *[_.buffer for _ in self.CRES]]))
        self.sizeValue = len(self.buffer)
        self.offsetValue = self.previous.offsetValue + self.previous.sizeValue
        creSizes = np.array([_.size for _ in self.CRES])
        creOffsets = np.concatenate([[0], creSizes]).cumsum()[:-1] + len(self.indexHeader) + self.offsetValue
        for o, s, ob, sb in zip(creOffsets, creSizes, self.CREOffsetBlocks, self.CRESizeBlocks):
            ob.bind_buffer(self.buffer).set_value(int(o))
            sb.bind_buffer(self.buffer).set_value(int(s))
        return self.buffer

In [1486]:
class Globals(RecordsBlock):
    
    Pattern = RecordBase(segList=[('NAME', 32, str)
                                 ,('TYPE', 2, int)
                                 ,('REF', 2, int)
                                 ,('DWORD', 4, int)
                                 ,('INT', 4, int)
                                 ,('DOUBEL', 8, int)
                                 ,('SCRIPT', 32, str)])
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)


In [1487]:
class KnownSpells(RecordsBlock):
    
    Pattern = RecordBase(segList=[('SPELL', 8, 'STRNAME')
                                 ,('LEVEL', 2, int)
                                 ,('TYPE', 2, int)])
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)


In [1488]:
class SpellMemorization(RecordsBlock):
    
    Pattern = RecordBase(segList=[('LEVEL', 2, int)
                                 ,('BASECOUNT', 2, int)
                                 ,('EFFCOUNT', 2, int)
                                 ,('TYPE', 2, int)
                                 ,('INDEX', 4, int)
                                 ,('COUNTMEM', 4, int)])
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    

In [1489]:
class MemorizedSpells(RecordsBlock):
    
    Pattern = RecordBase(segList=[('SPELL', 8,'STRNAME')
                                 ,('MEM', 4, int)])
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)


In [1490]:
class Effects(RecordsBlock):
    Pattern = RecordBase(segList=[('NULL', 8, int)
                                 ,('OPCODE', 4, 'INTNAME')
                                 ,('TARGET', 4, int)
                                 ,('POWER', 4, int)
                                 ,('PARAM1', 4, int)
                                 ,('PARAM2', 4, int)
                                 ,('TIME', 4, int)
                                 ,('OTHER', 232, None)]
                         ,template=bytearray([0, 0, 0, 0, 0, 0, 0, 0, 233, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 95, 
                                              0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                                              0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                                              0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                                              0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 255, 255, 255, 255, 255, 
                                              255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 0, 0, 0, 0, 0, 0, 0, 0, 
                                              0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 255, 255, 255, 255, 0, 0, 0, 0, 0, 0, 0, 0, 
                                              0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                                              0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                                              0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                                              0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.Pattern.nameMap = self.savRef.EFFECTCODES


In [1491]:
class Items(RecordsBlock):
    
    Pattern = RecordBase(segList=[('NAME', 8, 'STRNAME')
                                 ,('EXPIRE', 1, int)
                                 ,('ELAPSED', 1, int)
                                 ,('QUALITY1', 2, int)
                                 ,('QUALITY2', 2, int)
                                 ,('QUALITY3', 2, int)
                                 ,('IDENTIFIED', 1, int)
                                 ,('UNSTEALABLE', 1, int)
                                 ,('STOLEN', 1, int)
                                 ,('UNDROPPABLE', 1, int)])
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.Pattern.nameMap = self.savRef.ITEMCODES

In [1492]:
class ItemSlots(SegBlock):
    pass

In [1493]:
class GamCreBase(ABC, BlockBase):
    
    SEGMAP = {"KnownSpells": KnownSpells
             ,"SpellMemorization": SpellMemorization
             ,"MemorizedSpells": MemorizedSpells
             ,"Effects": Effects
             ,"Items": Items
             ,"ItemSlots": ItemSlots
             ,"Party": Party
             ,"NPC": Party
             ,"Global": Globals}
                    
        
    '''
    def initiate_seg_size(self):
        Retrieve offsets of each segment from segRecords to determine the size of each seg, 
        Write back into the SizeValue column
        And reset_index to ensure they are aligned in the right packing sequence
        offsets = self.segRecords[self.segRecords['OffsetLoc'].notnull()]
        offsets = offsets['OffsetLoc'].apply(lambda x: self.bytes2Num(self.buffer[self.to_int(x)
                                                                                 :self.to_int(x) + 4])).sort_values()
        nonZeroOffsets = pd.concat([pd.Series([0]), offsets[offsets!=0]])
        nonZeroOffsets[nonZeroOffsets > self.size] = self.size
        lengths = pd.concat([offsets[offsets==0], nonZeroOffsets.shift(-1, fill_value=self.size) - nonZeroOffsets])
        self.segRecords.update(lengths.sort_index().rename('SizeValue').to_frame())
        self.segRecords = (self.segRecords
                           .join(offsets.rename('OffsetValue').to_frame())
                           .fillna(value={'OffsetValue': 0})
                           .sort_values('OffsetValue')
                           .reset_index(drop=True))
    '''
        
        
    '''
    def get_CRE_locs(self):
        loc_df = self.savRef.creLocs
        CREIndexLen = loc_df[loc_df['SubSegs']=='Header']['SizeValue'].iloc[0]
        CREOffsetLoc, CRESizeLoc =  loc_df[loc_df['SubSegs']=='CRE'][['OffsetLoc', 'SizeLoc']].iloc[0].apply(self.to_int)
        return CREIndexLen, CREOffsetLoc, CRESizeLoc
    '''
        
        
    def init_seg_size(self):
        df = self.segRecords.copy()
        df.insert(df.shape[1]
                 ,'OffsetValue'
                 ,self.segRecords['OffsetLoc'].apply(lambda x: np.nan if pd.isnull(x) else self.bytes2Num(self.buffer[self.to_int(x)
                                                                                                                     :self.to_int(x)+4])))
        df1 = df[df['OffsetValue']!=0].fillna(value={'OffsetValue': 0})
        df2 = df[df['OffsetValue']==0]
        df1['SizeValue'] = df1['OffsetValue'].shift(-1, fill_value=self.size) - df1['OffsetValue']
        self.segRecords = pd.concat([df1, df2]).fillna(value={'SizeValue': 0}).sort_index()
        
    
    def init_values(self, buffer:Optional[bytearray]=None): 
        if not self.valueRecords.empty:
            if buffer == None:
                buffer = self.buffer
            self.VALUES = make_dataclass('VALUES', self.valueRecords.apply(lambda x: (x[0]
                                                                                     ,ValueBlock
                                                                                     ,ValueBlock(buffer, *x[1:]))
                                                                           ,axis=1))
            
    def init_tables(self, buffer:Optional[bytearray]=None):
        if not self.tableRecords.empty:
            if buffer == None:
                buffer = self.buffer
            self.TABLES = make_dataclass('TABLES', self.tableRecords.apply(lambda x: (x[0]
                                                                                     ,TableBlock
                                                                                     ,TableBlock(buffer
                                                                                                 ,*x[1:]
                                                                                                 ,getattr(self.savRef, x['RefTable']+'CODES')))
                                                                           ,axis=1))
                                                                           
            
    def init_segs(self, buffer:Optional[bytearray]=None):
        '''
        Need to keep sequence order
        So use pd.Series instead of dataclass
        '''
        if not self.segRecords.empty:
            if buffer == None:
                buffer = self.buffer
            self.SEGS = (self.segRecords
                         .set_index(self.segRecords.columns[0])
                         .apply(lambda x: self.SEGMAP.get(x.name, SegBlock)(self.savRef, self, buffer, x.name, *x), axis=1))
            for i in range(1, self.SEGS.size):
                self.SEGS[i].previous = self.SEGS[i-1]
            
            
    def pack(self):
        self.buffer = bytearray(b''.join([_.pack() for _ in self.SEGS]))
        self.size = len(self.buffer)
        for seg in self.SEGS:
            seg.offsetBlock.bind_buffer(self.buffer).set_value(seg.offsetValue)
            seg.countBlock.bind_buffer(self.buffer).set_value(seg.countValue)
            seg.sizeBlock.bind_buffer(self.buffer).set_value(seg.sizeValue)
        return self.buffer
    

In [1494]:
class GAM(GamCreBase):
    
    def __init__(self
                 ,savRef: object
                 ,buffer: bytearray):
        self.savRef = savRef
        self.resourceDir = savRef.resourceDir
        self.buffer = buffer
        self.size = len(self.buffer)
        self.valueRecords = savRef.gamValues
        self.segRecords = savRef.gamSegs
        self.VALUES = make_dataclass('SEGS', [])
        self.SEGS = pd.Series([], dtype=object)
        self.init_seg_size()
        self.init_values()
        self.init_segs()


In [1495]:
class CRE(GamCreBase):
    
    def __init__(self
                ,savRef:object
                ,parentRef:object
                ,buffer: bytearray
                ,valueRecords: Optional[pd.DataFrame] = None
                ,tableRecords: Optional[pd.DataFrame] = None
                ,segRecords: Optional[pd.DataFrame] = None
                ):
        self.savRef = savRef
        self.parentRef = parentRef
        self.buffer = buffer
        self.size = len(self.buffer)
        if valueRecords is None:
            self.valueRecords = pd.read_csv(os.path.join(self.savRef.resourceDir, 'CREVALUES.csv'), index_col=0)
        else:
            self.valueRecords = valueRecords.copy()
        if tableRecords is None:
            self.tableRecords = pd.read_csv(os.path.join(self.savRef.resourceDir, 'CRETABLES.csv'), index_col=0)
        else:
            self.tableRecords = tableRecords.copy()
        if segRecords is None:
            self.segRecords = pd.read_csv(os.path.join(self.savRef.resourceDir, 'CRESEGS.csv'), index_col=0)
        else:
            self.segRecords = segRecords.copy()
        self.VALUES = make_dataclass('SEGS', [])
        self.TABLES = make_dataclass('SEGS', [])
        self.SEGS = pd.Series([], dtype=object)
        self.init_seg_size()
        self.init_values()
        self.init_tables()
        self.init_segs()      
        

In [1496]:
class Sav(GamCreBase):
    
    def __init__(self, savefile:str, game:str='BGEE'):
        self.savefile = os.path.join(PKG_ROOT, 'saves', 'original', savefile)
        self.modified = os.path.join(PKG_ROOT, 'saves', 'modified', 'Edited_'+savefile)
        self.resourceDir = os.path.join(PKG_ROOT, 'resources', game)
        self.zipfile = zipfile.ZipFile(self.savefile)
        self.filelist = self.zipfile.namelist()
        self.files = [self.zipfile.read(_) for _ in self.filelist]
        self.gamStr = self.files[1].decode('latin')
        self.gamBuffer = bytearray(self.files[1])
        self.gamVersion = re.findall(r'GAME\s*V\d+\.\d+', self.gamStr)[0]  #always 'GAMEV2.0'
        self.creVersion = re.findall(r'CRE\s*V\d+\.\d+', self.gamStr)[0]  #always 'CRE V1.0'
        self.load_dfs()
        self.make_names()
        self.GAM = GAM(savRef=self, buffer=self.gamBuffer)
        self.Party = self.GAM.SEGS.Party.CRES
        self.NPC = self.GAM.SEGS.NPC.CRES
        
        
    def load_dfs(self):
        self.gamValues = pd.read_csv(os.path.join(self.resourceDir, 'GAMVALUES.csv'), index_col=0)
        self.gamSegs = pd.read_csv(os.path.join(self.resourceDir, 'GAMSEGS.csv'), index_col=0)
        self.creValues = pd.read_csv(os.path.join(self.resourceDir, 'CREVALUES.csv'), index_col=0)
        self.creTables = pd.read_csv(os.path.join(self.resourceDir, 'CRETABLES.csv'), index_col=0)
        self.creSegs = pd.read_csv(os.path.join(self.resourceDir, 'CRESEGS.csv'), index_col=0)
        self.creLocs = pd.read_csv(os.path.join(self.resourceDir, 'CRELOC.csv'), index_col=0)
    
    def make_names(self):
        for itm in ['ITEM', 'SPELL', 'EFFECT']:
            f = pd.read_csv(os.path.join(self.resourceDir, itm + '.csv'), index_col=0)
            setattr(self, itm, IDS.make_names(itm, f))
            setattr(self, itm + 'CODES', IDS.make_enum(itm+'CODES', f, nameCol=-1, valueCol=-2))
        for itm in ['WEAPON', 'PARTYSLOTS', 'NPCSLOTS', *self.creTables['RefTable'].drop_duplicates()]: #Race, Kit, Class, Gender, Alignment
            setattr(self, itm+'CODES', IDS.make_enum(itm+'CODES', pd.read_csv(os.path.join(self.resourceDir, itm + '.csv'), index_col=0)))
            
            
    def pack(self, filename:str=None):
        if filename is None:
            filename = self.modified
        self.zipfile.close()
        with zipfile.ZipFile(filename, 'w') as target:
            target.writestr(self.filelist[0], self.files[0], compress_type=zipfile.ZIP_STORED)
            target.writestr(self.filelist[1], self.GAM.pack(), compress_type=zipfile.ZIP_STORED)
            for _name, _f in zip(self.filelist[2:], self.files[2:]):
                target.writestr(_name, _f, compress_type=zipfile.ZIP_STORED)
            
        

In [1497]:
s = Sav('TESTSAVE.bg1save', 'BGEE')

In [1498]:
s.GAM.SEGS.Global

,NAME,TYPE,REF,DWORD,INT,DOUBEL,SCRIPT
0,TUTORIALITEMCHECK,0,0,0,1,0,                                
1,ACH_UNTOUCHABLE,0,0,0,1,0,                                
2,DREAM,0,0,0,4294967295,0,                                
3,CHAPTER,0,0,0,0,0,                                
4,BD_CHAPTER_SAVE,0,0,0,4294967295,0,                                
5,BD_NIGHTMARE_MODE,0,0,0,1,0,                                
6,TETHTORIL,0,0,0,32399,0,                                
7,ACH_GODLIKE,0,0,0,1,0,                                
8,GAMEDIFFICULTY,0,0,0,7,0,                                
9,2643DOOR,0,0,0,1,0,                                


<class '__main__.Globals'> Global

In [1505]:
s.Party[0].SEGS.Effects

,NULL,OPCODE,TARGET,POWER,PARAM1,PARAM2,TIME,OTHER
0,0,Stat_Proficiency_Modifier,0,0,2,113,9,b'\x00\x00\x00\x00d\x00\x00\x00\x00\x00\x00\x0...
1,0,Stat_Proficiency_Modifier,0,0,2,112,9,b'\x00\x00\x00\x00d\x00\x00\x00\x00\x00\x00\x0...
2,0,Stat_Proficiency_Modifier,0,0,2,111,9,b'\x00\x00\x00\x00d\x00\x00\x00\x00\x00\x00\x0...
3,0,Stat_Proficiency_Modifier,0,0,5,107,9,b'\x00\x00\x00\x00d\x00\x00\x00\x00\x00\x00\x0...
4,0,Stat_Proficiency_Modifier,0,0,5,106,9,b'\x00\x00\x00\x00d\x00\x00\x00\x00\x00\x00\x0...
5,0,Stat_Proficiency_Modifier,0,0,5,105,9,b'\x00\x00\x00\x00d\x00\x00\x00\x00\x00\x00\x0...
6,0,Stat_Proficiency_Modifier,0,0,5,104,9,b'\x00\x00\x00\x00d\x00\x00\x00\x00\x00\x00\x0...
7,0,Stat_Proficiency_Modifier,0,0,5,103,9,b'\x00\x00\x00\x00d\x00\x00\x00\x00\x00\x00\x0...
8,0,Stat_Proficiency_Modifier,0,0,5,102,9,b'\x00\x00\x00\x00d\x00\x00\x00\x00\x00\x00\x0...
9,0,Stat_Proficiency_Modifier,0,0,5,99,9,b'\x00\x00\x00\x00d\x00\x00\x00\x00\x00\x00\x0...


<class '__main__.Effects'> Effects

In [1500]:
p = s.GAM.SEGS.Global.Pattern.pattern

In [1501]:
t = s.GAM.SEGS.Global.Pattern.template

In [1502]:
d = {'NAME': b'ABCD', 'INT': b'5'}

In [1503]:
re.sub(p, b'{}', t)

b'{}'

In [1504]:
p.search(t).groups()

(b'00000000000000000000000000000000',
 b'00',
 b'00',
 b'0000',
 b'0000',
 b'00000000',
 b'00000000000000000000000000000000')